# '학군' 이 좋으면, 부동산 가격이 높을까?

## 가설

학군이 좋으면, 부동산 매매 가격이 상대적으로 높을 것이다.

## 정의(def)

* 학군이란?
    * 중학교에서 특수목적고로 진학하는 비율
* 부동산 매매 가격?
    * 아파트 거래가격

# 1. 데이터 가져오기

In [4]:
# 1. 원본 데이터
import pandas as pd
graduate_path = f'data/part1_2018년_2차_졸업생의_진로_현황(전체).xlsx'
raw_graduate = pd.read_excel(graduate_path, sheet_name ='2018_졸업생의 진로 현황(중)')

In [5]:
print(graduate_path)

data/part1_2018년_2차_졸업생의_진로_현황(전체).xlsx


# 2. 전처리
## 2-1. 원본 데이터에서 필요한 데이터만 추출

In [11]:
# 2. 전처리
# 전처리는 데이터를 분석하기전에 필요한 데이터를 수집하고, 보정하는 행위
# 전처리는 {누락데이터}를 처리하는것이 대부분이고, 수치화(항목, 속성, 목록 ...)도 함께 함
# ''지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2'
type(raw_graduate)

pandas.core.frame.DataFrame

In [17]:
select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
graduate_data = select_col.drop(0)
graduate_data
# 누락 데이터 검색 방법
# 1) 졸업자 컬럼의 행을 하나씩 읽어와서 type 이 숫자가 아니면 0으로 바꾼다.
# 2) 졸업자 컬럼의 행을 하나씩 읽어와서 숫자가 있는 행만 불러옴.
# 3) 숫자로 변경해주는 기능이 있는가?

# 2-2. 수치화
graduate_data['졸업자'] = pd.to_numeric(graduate_data['졸업자'])
graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
graduate_data['외고'] = pd.to_numeric(graduate_data['외고'])
graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
graduate_data

,지역,학교명,학교코드,졸업자,과고,외고,총합
1,서울특별시 성북구,서울대학교사범대학부설중학교,S000003514,234.0,4.0,5.0,9.0
2,서울특별시 종로구,서울대학교사범대학부설여자중학교,S000003562,172.0,0.0,8.0,8.0
3,서울특별시 강남구,개원중학교,S010000698,214.0,2.0,9.0,11.0
4,서울특별시 서초구,경원중학교,S010000700,310.0,5.0,4.0,9.0
5,서울특별시 강남구,구룡중학교,S010000701,256.0,3.0,4.0,7.0
...,...,...,...,...,...,...,...
3454,제주특별자치도 제주시,탐라중학교,S170000844,298.0,1.0,1.0,2.0
3455,제주특별자치도 제주시,제주영지학교,S170000512,12.0,0.0,0.0,0.0
3456,제주특별자치도 제주시,제주영송학교,S170000513,21.0,0.0,0.0,0.0
3457,제주특별자치도 서귀포시,서귀포온성학교,S170000790,9.0,0.0,0.0,0.0


In [18]:
# [지역] 컬럼을 기준으로 데이터를 읽어야 됩니다.
# temp = "서울특별시 성북구"
# 공백(spacce)을 기준으로 문자열을 분리
# 서울특별시를 index = 0, 성북구를 index = 1
#graduate_data[['시도'], ['구군']]

In [32]:
province = "경상북도 성북구"
temp = province.split(' ')
sido = temp[0]
if len(sido) != 4:
        print(sido[:2])
else:
       print(sido[0] + sido[2])

경북


In [34]:
# 지역을 '시도', '구군'으로 분리
# 입력은 "서울특별시 성북구" 그리고 "경상북도", "경상남도" 와 같은 시도도 입력됨
# 처리
# 1) 시와 구를ㄹ 나눈다. (공백으로 나눌 수 있는가?) => split
# 2) 만약에 시를 구눕했을 때, 글자수가 4자이면 첫번째와 세번째 글자를 합쳐서 출력 => 경남 [0] + [2]
# 3) 글자수가 4자가 아니면 앞의 두 글자만 출력 => [0:2]
# 출력은 "서울" or "경남"

In [39]:
print(len(province))

8


In [3]:
def get_province(x):
    temp = x.split(' ')[0]
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0] + temp[2]

graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_province)
graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])

# 2-2
graduate_data['시도'].isnull().sum()
graduate_data.index[graduate_data['시도'].isnull()]

#2-3
graduate_data.loc[588]
graduate_data.loc[3011]
# 신정중학교
graduate_data.at[588,'시도'] = '부산' 
graduate_data.at[588,'구군'] = '기장군'
# 은풍중학교
graduate_data.at[3011,'시도'] = '경북'
graduate_data.at[3011,'구군'] = '예천군'

NameError: name 'graduate_data' is not defined

In [1]:
# 5-1. 그래프 준비
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 5-2. 그래프 그려보기
plt.figure(figsize=(10, 7))
plt.scatter(gradu_sido.index, gradu_sido['진학률'])
plt.xlabel('시도')
plt.ylabel('진학률')        
plt.show()

NameError: name 'gradu_sido' is not defined

<Figure size 720x504 with 0 Axes>

In [5]:
def fib2(n):
  result = []
  a, b = 0, 1
  while a < n:
    result.append(a)
    a, b = b, a+b
    return result

f100 = fib2(100)
print(f100)

[0]


In [6]:
def quadratic1(a,b,c):
    D = ((b**2)-4*a*c)**0.5
    X1 = (-b + D) / 2*a
    X2 = (-b - D) / 2*a
    print('X1:{}, X2:{}'.format(X1,X2))

quadratic1(2,2,2)

X1:(-1.9999999999999998+3.4641016151377544j), X2:(-2-3.4641016151377544j)
